# Approximation du Nombre $\pi$ par des Méthodes de Machine Learning

**M1 Informatique — Spécialité Intelligence Artificielle**  
**Mathématiques Appliquées au Machine Learning**

---

Ce notebook implémente et compare quatre approches d'approximation de $\pi$ :

1. **Monte-Carlo stochastique** — estimateur empirique basé sur la loi des grands nombres
2. **Régression polynomiale** — approximation fonctionnelle par moindres carrés
3. **Réseau de neurones MLP** — approximation universelle (Cybenko 1989, Hornik 1991)
4. **Processus Gaussien** — inférence bayésienne non-paramétrique (Rasmussen & Williams 2006)

**Dataset** : données synthétiques générées depuis la relation géométrique $y = \sqrt{1-x^2}$,  
de sorte que $\pi = 4\int_0^1 \sqrt{1-x^2}\,dx$.


In [ ]:
# ─── Imports ─────────────────────────────────────────────────────────────────
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import rcParams
import warnings
warnings.filterwarnings('ignore')

from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

import time

# ─── Configuration matplotlib ────────────────────────────────────────────────
rcParams['figure.dpi'] = 130
rcParams['font.family'] = 'DejaVu Sans'
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

COLORS = {
    'mc':   '#1f4e79',   # Navy Blue
    'poly': '#1d6b2e',   # Forest Green
    'mlp':  '#8b0000',   # Dark Red
    'gp':   '#d4700a',   # Orange
    'true': '#555555',   # Gray
}

PI = np.pi
print(f"Valeur de référence : π = {PI:.15f}")
print(f"Packages chargés avec succès ✓")

---
## 1. Génération du Dataset Synthétique

**Principe géométrique :** L'aire d'un demi-disque unitaire est $\pi/2$, soit :
$$\int_0^1 \sqrt{1-x^2}\,dx = \frac{\pi}{4}$$

Nous observons la fonction $f(x) = \sqrt{1-x^2}$ en $n$ points avec bruit gaussien :
$$y_i = \sqrt{1-x_i^2} + \varepsilon_i, \quad \varepsilon_i \sim \mathcal{N}(0, \sigma^2)$$


In [ ]:
def generate_dataset(n: int, sigma_noise: float = 0.01, seed: int = 42) -> tuple:
    """
    Génère le dataset synthétique basé sur le quart de cercle.
    
    Paramètres
    ----------
    n            : nombre de points
    sigma_noise  : écart-type du bruit gaussien
    seed         : graine aléatoire pour reproductibilité
    
    Retourne
    --------
    X : array (n,) — abscisses dans [0, 1]
    y : array (n,) — ordonnées bruitées
    y_true : array (n,) — ordonnées exactes
    """
    rng = np.random.default_rng(seed)
    X = np.sort(rng.uniform(0, 1, n))
    y_true = np.sqrt(1 - X**2)
    y = y_true + rng.normal(0, sigma_noise, n)
    return X, y, y_true

# Dataset principal
N_TRAIN = 300
SIGMA    = 0.01
X_train, y_train, y_clean = generate_dataset(N_TRAIN, SIGMA)

# Grille de test dense
X_test  = np.linspace(0, 1, 1000)
y_exact = np.sqrt(1 - X_test**2)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# (a) Données + courbe exacte
ax = axes[0]
ax.scatter(X_train, y_train, s=8, alpha=0.5, color=COLORS['mc'], label=f'Données bruitées ($n={N_TRAIN}$, $\sigma={SIGMA}$)')
ax.plot(X_test, y_exact, color=COLORS['true'], lw=2, label=r'$f(x)=\sqrt{1-x^2}$ (exacte)')
ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$y$', fontsize=12)
ax.set_title('Dataset synthétique : quart de cercle', fontweight='bold')
ax.legend(fontsize=9)
ax.set_aspect('equal')

# (b) Interprétation géométrique
ax = axes[1]
theta = np.linspace(0, np.pi/2, 500)
ax.fill_between(np.cos(theta), np.sin(theta), alpha=0.15, color=COLORS['gp'], label=f'Aire = $\pi/4 \\approx$ {PI/4:.4f}')
ax.plot(np.cos(theta), np.sin(theta), color=COLORS['mc'], lw=2.5, label='Quart de cercle')
ax.plot([0,1,1,0,0],[0,0,1,1,0], 'k--', lw=1, alpha=0.5)
ax.set_aspect('equal')
ax.set_xlim(-0.05, 1.1); ax.set_ylim(-0.05, 1.1)
ax.set_xlabel('$x$', fontsize=12); ax.set_ylabel('$y$', fontsize=12)
ax.set_title(r'$\int_0^1\sqrt{1-x^2}\,dx = \pi/4$', fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('fig_dataset.png', bbox_inches='tight')
plt.show()
print(f"Dataset : {N_TRAIN} points, SNR ≈ {(1/(2*SIGMA)):.1f}")

---
## 2. Méthode 1 : Monte-Carlo Stochastique

**Estimateur :**
$$\hat{\pi}_n^{\text{MC}} = \frac{4}{n}\sum_{i=1}^n \mathbf{1}_{U_{1,i}^2 + U_{2,i}^2 \leq 1}, \quad (U_1,U_2)\sim\mathcal{U}([0,1]^2)$$

**Convergence (TCL) :** $\sqrt{n}(\hat{\pi}_n - \pi) \xrightarrow{\mathcal{D}} \mathcal{N}(0,\, \pi(4-\pi))$

**Taux :** $\mathbb{E}[(\hat{\pi}_n - \pi)^2] = O(n^{-1})$, erreur en $O(n^{-1/2})$


In [ ]:
def monte_carlo_pi(n: int, seed: int = 0) -> tuple:
    """
    Estime π par la méthode de Monte-Carlo.
    Retourne : (estimation, erreur_std, points_dans_cercle, tous_les_points)
    """
    rng = np.random.default_rng(seed)
    pts = rng.uniform(0, 1, (n, 2))
    inside = (pts[:, 0]**2 + pts[:, 1]**2) <= 1.0
    pi_hat = 4 * inside.mean()
    # IC 95% asymptotique
    p_hat = inside.mean()
    std_pi = 4 * np.sqrt(p_hat * (1 - p_hat) / n)
    return pi_hat, std_pi, inside, pts

# ── Convergence sur différentes tailles ─────────────────────────────────────
ns = np.logspace(1, 6, 60, dtype=int)
pi_mc_list, std_mc_list = [], []

for n in ns:
    pi_hat, std, _, _ = monte_carlo_pi(n, seed=42)
    pi_mc_list.append(pi_hat)
    std_mc_list.append(std)

pi_mc_arr = np.array(pi_mc_list)
std_mc_arr = np.array(std_mc_list)
errors_mc  = np.abs(pi_mc_arr - PI)

# ── Visualisation ────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# (a) Points Monte-Carlo (n=5000)
ax = axes[0]
pi_demo, _, inside_demo, pts_demo = monte_carlo_pi(5000, seed=1)
ax.scatter(pts_demo[inside_demo, 0], pts_demo[inside_demo, 1],
           s=1.5, alpha=0.4, color=COLORS['mc'], label='Dans le cercle')
ax.scatter(pts_demo[~inside_demo, 0], pts_demo[~inside_demo, 1],
           s=1.5, alpha=0.4, color='#e05c5c', label='Hors du cercle')
theta = np.linspace(0, np.pi/2, 300)
ax.plot(np.cos(theta), np.sin(theta), 'k-', lw=2)
ax.set_aspect('equal')
ax.set_title(f'Monte-Carlo (n=5000)\n$\hat{{\\pi}}$ = {pi_demo:.5f}', fontweight='bold')
ax.legend(fontsize=8, markerscale=4)

# (b) Convergence log-log
ax = axes[1]
ax.loglog(ns, errors_mc, color=COLORS['mc'], lw=1.5, label='Erreur MC')
ax.fill_between(ns, np.maximum(errors_mc - std_mc_arr, 1e-8),
                errors_mc + std_mc_arr, alpha=0.2, color=COLORS['mc'])
# Droite théorique O(n^-0.5)
ref_x = np.array([ns[0], ns[-1]])
ax.loglog(ref_x, 2.0 * ref_x**(-0.5), 'k--', lw=1.5, label=r'$O(n^{-1/2})$')
ax.set_xlabel('$n$ (nombre de points)', fontsize=11)
ax.set_ylabel('$|\hat{\\pi}_n - \pi|$', fontsize=11)
ax.set_title('Convergence Monte-Carlo', fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# (c) Distribution de l'estimateur (TCL)
ax = axes[2]
N_SIMU = 2000
n_fixed = 1000
pi_samples = [monte_carlo_pi(n_fixed, seed=s)[0] for s in range(N_SIMU)]
pi_samples = np.array(pi_samples)
ax.hist(pi_samples, bins=50, density=True, color=COLORS['mc'],
        alpha=0.7, edgecolor='white', label=f'$n={n_fixed}$, {N_SIMU} tirages')
# Densité théorique (TCL)
sigma_th = np.sqrt(PI * (4 - PI) / n_fixed)
x_gauss  = np.linspace(pi_samples.min(), pi_samples.max(), 300)
from scipy.stats import norm
ax.plot(x_gauss, norm.pdf(x_gauss, PI, sigma_th), 'r-', lw=2,
        label=r'$\mathcal{N}(\pi,\, \pi(4-\pi)/n)$ (TCL)')
ax.axvline(PI, color='k', lw=1.5, linestyle='--', label=f'$\pi$ = {PI:.5f}')
ax.set_xlabel('$\hat{\\pi}_n$', fontsize=11)
ax.set_ylabel('Densité', fontsize=11)
ax.set_title('Distribution de $\hat{\\pi}_n$ (validation TCL)', fontweight='bold')
ax.legend(fontsize=8)

plt.tight_layout()
plt.savefig('fig_montecarlo.png', bbox_inches='tight')
plt.show()

pi_mc_final, std_final, _, _ = monte_carlo_pi(100_000)
print(f"Monte-Carlo (n=100,000) : π̂ = {pi_mc_final:.8f}")
print(f"  Erreur absolue : {abs(pi_mc_final - PI):.2e}")
print(f"  IC 95% : [{pi_mc_final - 1.96*std_final:.6f}, {pi_mc_final + 1.96*std_final:.6f}]")

---
## 3. Méthode 2 : Régression Polynomiale

**Principe :** Approximer $f(x) = \sqrt{1-x^2}$ par un polynôme $p_d(x) = \sum_{k=0}^d \theta_k x^k$, puis estimer $\pi$ par intégration :
$$\hat{\pi}_{\text{poly}} = 4\int_0^1 p_d(x)\,dx = 4\sum_{k=0}^d \frac{\theta_k}{k+1}$$

**Estimateur OLS :** $\hat{\bm{\theta}} = (\Phi^\top\Phi + \lambda I)^{-1}\Phi^\top y$ (régression Ridge)

**Sélection du degré :** Validation croisée $K$-fold minimisant l'erreur de généralisation.


In [ ]:
class PolynomialPiEstimator:
    """
    Estimateur de π par régression polynomiale Ridge.
    
    Attributs
    ---------
    degree  : degré polynomial optimal
    alpha   : paramètre de régularisation Ridge
    pipeline : sklearn Pipeline (PolynomialFeatures + Ridge)
    pi_hat  : estimation de π
    theta   : coefficients ajustés
    """
    
    def __init__(self, max_degree: int = 15, alpha: float = 1e-10):
        self.max_degree = max_degree
        self.alpha = alpha
        self.best_degree = None
        self.pipeline = None
        self.cv_scores = {}
    
    def select_degree_cv(self, X: np.ndarray, y: np.ndarray, cv: int = 5) -> int:
        """Sélection du degré optimal par validation croisée K-fold."""
        degrees = range(2, self.max_degree + 1)
        for d in degrees:
            pipe = Pipeline([
                ('poly', PolynomialFeatures(degree=d, include_bias=True)),
                ('ridge', Ridge(alpha=self.alpha))
            ])
            scores = cross_val_score(pipe, X.reshape(-1,1), y,
                                     cv=cv, scoring='neg_mean_squared_error')
            self.cv_scores[d] = -scores.mean()
        self.best_degree = min(self.cv_scores, key=self.cv_scores.get)
        return self.best_degree
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> 'PolynomialPiEstimator':
        """Entraîne le modèle et estime π par intégration analytique."""
        if self.best_degree is None:
            self.select_degree_cv(X, y)
        
        self.pipeline = Pipeline([
            ('poly', PolynomialFeatures(degree=self.best_degree, include_bias=True)),
            ('ridge', Ridge(alpha=self.alpha))
        ])
        self.pipeline.fit(X.reshape(-1,1), y)
        
        # Coefficients θ_k : p_d(x) = Σ θ_k x^k
        self.theta = self.pipeline.named_steps['ridge'].coef_.copy()
        self.theta[0] = self.pipeline.named_steps['ridge'].intercept_
        
        # Intégrale analytique : ∫₀¹ Σ θ_k x^k dx = Σ θ_k / (k+1)
        k_vals = np.arange(len(self.theta))
        self.pi_hat = 4.0 * np.sum(self.theta / (k_vals + 1))
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return self.pipeline.predict(X.reshape(-1,1))


# ── Entraînement et sélection de degré ──────────────────────────────────────
t0 = time.time()
poly_est = PolynomialPiEstimator(max_degree=15, alpha=1e-12)
poly_est.fit(X_train, y_train)
t_poly = time.time() - t0

print(f"Degré optimal sélectionné (CV-5) : d = {poly_est.best_degree}")
print(f"π̂_poly = {poly_est.pi_hat:.10f}")
print(f"Erreur absolue : {abs(poly_est.pi_hat - PI):.2e}")
print(f"Temps : {t_poly:.3f}s")

# ── Visualisation ────────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# (a) Ajustement polynomial
ax = axes[0]
y_pred_poly = poly_est.predict(X_test)
ax.scatter(X_train, y_train, s=8, alpha=0.4, color=COLORS['poly'], zorder=2, label='Données')
ax.plot(X_test, y_exact, color=COLORS['true'], lw=2, label='$f(x)$ exacte')
ax.plot(X_test, y_pred_poly, color=COLORS['poly'], lw=2, linestyle='--',
        label=f'Poly. d={poly_est.best_degree}')
ax.set_xlabel('$x$', fontsize=11); ax.set_ylabel('$y$', fontsize=11)
ax.set_title(f'Régression polynomiale (d={poly_est.best_degree})', fontweight='bold')
ax.legend(fontsize=9)

# (b) Courbe MSE vs degré (compromis biais-variance)
ax = axes[1]
degrees = list(poly_est.cv_scores.keys())
mse_vals = [poly_est.cv_scores[d] for d in degrees]
ax.semilogy(degrees, mse_vals, 'o-', color=COLORS['poly'], lw=2, ms=5)
ax.axvline(poly_est.best_degree, color='red', lw=1.5, linestyle='--',
           label=f'Optimal d={poly_est.best_degree}')
ax.set_xlabel('Degré $d$', fontsize=11)
ax.set_ylabel('MSE CV (log)', fontsize=11)
ax.set_title('Sélection de degré par validation croisée', fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# (c) Convergence vs n
ax = axes[2]
ns_poly = [50, 100, 200, 400, 700, 1000, 2000, 5000]
err_poly = []
for n in ns_poly:
    Xp, yp, _ = generate_dataset(n, SIGMA, seed=42)
    est = PolynomialPiEstimator(max_degree=12, alpha=1e-12)
    est.fit(Xp, yp)
    err_poly.append(abs(est.pi_hat - PI))

ax.loglog(ns_poly, err_poly, 's-', color=COLORS['poly'], lw=2, ms=6, label='Poly (d optimal)')
ref_x = np.array([ns_poly[0], ns_poly[-1]])
ax.loglog(ref_x, 5e-2 * (ref_x/ns_poly[0])**(-1.5), 'k--', lw=1.5, label='$O(n^{-1.5})$')
ax.set_xlabel('$n$ (taille du dataset)', fontsize=11)
ax.set_ylabel('$|\hat{\\pi} - \pi|$', fontsize=11)
ax.set_title('Convergence de la régression polynomiale', fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('fig_polynomial.png', bbox_inches='tight')
plt.show()

---
## 4. Méthode 3 : Réseau de Neurones MLP (from scratch)

**Théorème d'approximation universelle (Cybenko 1989) :**  
Pour toute $f \in C([0,1])$ et tout $\varepsilon > 0$, $\exists$ MLP à 1 couche cachée tel que $\|f_{\theta} - f\|_\infty < \varepsilon$.

**Architecture :** $x \in \mathbb{R} \to [64] \to [64] \to [32] \to \mathbb{R}$, activation GELU

**Optimisation :** Adam avec $\eta=5\times10^{-4}$, $\beta_1=0.9$, $\beta_2=0.999$

**Extraction de π :** $\hat{\pi}_{\text{MLP}} = 4 \times \frac{1}{M}\sum_{j=1}^M f_\theta(x_j^*)$ (règle des trapèzes)


In [ ]:
# ── Implémentation MLP NumPy pur (sans torch/tensorflow) ─────────────────────

def gelu(x):
    """Gaussian Error Linear Unit : x * Φ(x)"""
    return x * 0.5 * (1 + np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * x**3)))

def gelu_grad(x):
    """Dérivée de GELU."""
    tanh_val = np.tanh(np.sqrt(2/np.pi) * (x + 0.044715 * x**3))
    dtanh = 1 - tanh_val**2
    d_inner = np.sqrt(2/np.pi) * (1 + 3 * 0.044715 * x**2)
    return 0.5 * (1 + tanh_val) + x * 0.5 * dtanh * d_inner


class MLP:
    """
    Réseau de neurones MLP implémenté en NumPy pur.
    
    Architecture : Input(1) -> Hidden layers -> Output(1)
    Activation : GELU (couches cachées) + identité (sortie)
    """
    
    def __init__(self, layer_sizes: list, seed: int = 0):
        """
        Paramètres
        ----------
        layer_sizes : liste des dimensions, ex. [1, 64, 64, 32, 1]
        """
        rng = np.random.default_rng(seed)
        self.layer_sizes = layer_sizes
        self.L = len(layer_sizes) - 1  # nombre de couches
        
        # Initialisation He (Kaiming) : var(W) = 2/n_in
        self.W = []
        self.b = []
        for i in range(self.L):
            n_in, n_out = layer_sizes[i], layer_sizes[i+1]
            std = np.sqrt(2.0 / n_in)
            self.W.append(rng.normal(0, std, (n_in, n_out)))
            self.b.append(np.zeros(n_out))
        
        # État Adam
        self.mW = [np.zeros_like(w) for w in self.W]
        self.vW = [np.zeros_like(w) for w in self.W]
        self.mb = [np.zeros_like(b) for b in self.b]
        self.vb = [np.zeros_like(b) for b in self.b]
        self.t  = 0
    
    def forward(self, X: np.ndarray) -> tuple:
        """
        Propagation avant.
        Retourne : (output, liste des pre-activations, liste des activations)
        """
        zs, as_ = [], []
        a = X.reshape(-1, 1) if X.ndim == 1 else X
        for i in range(self.L - 1):
            z = a @ self.W[i] + self.b[i]
            zs.append(z)
            a = gelu(z)
            as_.append(a)
        # Couche de sortie (linéaire)
        z = a @ self.W[-1] + self.b[-1]
        zs.append(z)
        as_.append(z)  # identité
        return z, zs, as_
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        out, _, _ = self.forward(X)
        return out.flatten()
    
    def backward(self, X: np.ndarray, y: np.ndarray) -> tuple:
        """Rétropropagation du gradient (perte MSE)."""
        n = len(y)
        X_in = X.reshape(-1, 1) if X.ndim == 1 else X
        y_hat, zs, as_ = self.forward(X_in)
        
        loss = np.mean((y_hat.flatten() - y)**2)
        
        # Gradient couche de sortie
        delta = 2 * (y_hat.flatten() - y).reshape(-1, 1) / n  # (n, 1)
        
        dW = [None] * self.L
        db = [None] * self.L
        
        # Couche de sortie
        a_prev = as_[-2] if self.L > 1 else X_in
        dW[-1] = a_prev.T @ delta
        db[-1] = delta.sum(axis=0)
        
        # Couches cachées (de L-2 à 0)
        for i in range(self.L - 2, -1, -1):
            delta = (delta @ self.W[i+1].T) * gelu_grad(zs[i])
            a_prev = as_[i-1] if i > 0 else X_in
            dW[i] = a_prev.T @ delta
            db[i] = delta.sum(axis=0)
        
        return loss, dW, db
    
    def adam_step(self, dW: list, db: list, lr: float = 5e-4,
                  beta1: float = 0.9, beta2: float = 0.999, eps: float = 1e-8):
        """Mise à jour Adam (Kingma & Ba, 2014)."""
        self.t += 1
        for i in range(self.L):
            # Moments poids
            self.mW[i] = beta1 * self.mW[i] + (1 - beta1) * dW[i]
            self.vW[i] = beta2 * self.vW[i] + (1 - beta2) * dW[i]**2
            mW_hat = self.mW[i] / (1 - beta1**self.t)
            vW_hat = self.vW[i] / (1 - beta2**self.t)
            self.W[i] -= lr * mW_hat / (np.sqrt(vW_hat) + eps)
            # Moments biais
            self.mb[i] = beta1 * self.mb[i] + (1 - beta1) * db[i]
            self.vb[i] = beta2 * self.vb[i] + (1 - beta2) * db[i]**2
            mb_hat = self.mb[i] / (1 - beta1**self.t)
            vb_hat = self.vb[i] / (1 - beta2**self.t)
            self.b[i] -= lr * mb_hat / (np.sqrt(vb_hat) + eps)
    
    def train(self, X: np.ndarray, y: np.ndarray,
              n_epochs: int = 3000, batch_size: int = 64,
              lr: float = 5e-4, verbose_every: int = 500) -> list:
        """Boucle d'entraînement SGD-Adam avec mini-batches."""
        n = len(X)
        rng = np.random.default_rng(99)
        losses = []
        for epoch in range(n_epochs):
            # Shuffle
            idx = rng.permutation(n)
            X_shuf, y_shuf = X[idx], y[idx]
            epoch_loss = 0.0
            n_batches  = 0
            for start in range(0, n, batch_size):
                Xb = X_shuf[start:start+batch_size]
                yb = y_shuf[start:start+batch_size]
                loss, dW, db = self.backward(Xb, yb)
                self.adam_step(dW, db, lr=lr)
                epoch_loss += loss
                n_batches  += 1
            losses.append(epoch_loss / n_batches)
            if (epoch + 1) % verbose_every == 0:
                print(f"  Époque {epoch+1:4d}/{n_epochs} — Loss MSE : {losses[-1]:.6f}")
        return losses
    
    def estimate_pi(self, M: int = 10000) -> float:
        """Estime π par règle des trapèzes sur la prédiction du réseau."""
        x_quad = np.linspace(0, 1, M)
        f_pred  = self.predict(x_quad)
        return 4.0 * np.trapz(f_pred, x_quad)


# ── Entraînement ─────────────────────────────────────────────────────────────
print("Entraînement du MLP...")
t0 = time.time()
mlp = MLP(layer_sizes=[1, 64, 64, 32, 1], seed=7)
losses = mlp.train(X_train, y_train, n_epochs=3000, batch_size=64, lr=5e-4, verbose_every=1000)
t_mlp = time.time() - t0

pi_mlp = mlp.estimate_pi(M=50000)
print(f"\nRésultats MLP :")
print(f"  π̂_MLP = {pi_mlp:.10f}")
print(f"  Erreur absolue : {abs(pi_mlp - PI):.2e}")
print(f"  Temps d'entraînement : {t_mlp:.1f}s")

In [ ]:
# ── Visualisation MLP ─────────────────────────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# (a) Courbe d'entraînement
ax = axes[0]
ax.semilogy(losses, color=COLORS['mlp'], lw=1.5, alpha=0.8)
ax.set_xlabel('Époque', fontsize=11)
ax.set_ylabel('MSE Loss (log)', fontsize=11)
ax.set_title('Courbe de perte (Adam)', fontweight='bold')
ax.grid(True, alpha=0.3)
ax.text(0.65, 0.7, f'Loss finale\n{losses[-1]:.2e}', transform=ax.transAxes,
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5), fontsize=9)

# (b) Ajustement MLP
ax = axes[1]
y_pred_mlp = mlp.predict(X_test)
ax.scatter(X_train, y_train, s=8, alpha=0.4, color=COLORS['mlp'], zorder=2, label='Données')
ax.plot(X_test, y_exact, color=COLORS['true'], lw=2.5, label='$f(x)$ exacte')
ax.plot(X_test, y_pred_mlp, '--', color=COLORS['mlp'], lw=2,
        label=f'MLP (3 couches)')
residuals = y_pred_mlp - y_exact
ax.fill_between(X_test, y_exact + residuals, y_exact, alpha=0.3, color=COLORS['mlp'], label='Résidu')
ax.set_xlabel('$x$', fontsize=11); ax.set_ylabel('$y$', fontsize=11)
ax.set_title(f'Ajustement MLP\n$\hat{{\\pi}}$={pi_mlp:.6f}', fontweight='bold')
ax.legend(fontsize=9)

# (c) Distribution des résidus
ax = axes[2]
ax.hist(residuals, bins=40, density=True, color=COLORS['mlp'], alpha=0.7,
        edgecolor='white', label='Résidus MLP')
x_res = np.linspace(residuals.min(), residuals.max(), 200)
from scipy.stats import norm as norm_dist
ax.plot(x_res, norm_dist.pdf(x_res, residuals.mean(), residuals.std()),
        'r-', lw=2, label=r'$\mathcal{N}(\mu,\sigma^2)$ ajustée')
ax.axvline(0, color='k', lw=1.5, linestyle='--')
ax.set_xlabel('Résidu $f_{\\theta}(x) - f(x)$', fontsize=11)
ax.set_ylabel('Densité', fontsize=11)
ax.set_title(f'Distribution des résidus\n($\mu$={residuals.mean():.2e}, $\sigma$={residuals.std():.2e})',
             fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('fig_mlp.png', bbox_inches='tight')
plt.show()

---
## 5. Méthode 4 : Processus Gaussien (Inférence Bayésienne)

**Modèle :** $f \sim \mathcal{GP}(0, k_{\text{RBF}})$ avec $k_{\text{RBF}}(x,x') = \sigma_f^2 \exp\!\left(-\frac{(x-x')^2}{2\ell^2}\right)$

**Postérieure exacte :**
$$\mu_*(x_*) = \mathbf{k}_*^\top (\mathbf{K} + \sigma_n^2 I)^{-1} \mathbf{y}, \qquad \sigma_*^2(x_*) = k(x_*,x_*) - \mathbf{k}_*^\top (\mathbf{K} + \sigma_n^2 I)^{-1} \mathbf{k}_*$$

**Hyperparamètres :** Optimisés par maximisation de la log-vraisemblance marginale (MLE-II)


In [ ]:
class GaussianProcessRegressor:
    """
    Régression par Processus Gaussien — noyau RBF (Matérn ∞).
    Implémentation NumPy avec optimisation MLE-II des hyperparamètres.
    
    Paramètres
    ----------
    sigma_f  : amplitude du noyau (signal std)
    length_scale : échelle de longueur ℓ
    sigma_n  : bruit d'observation
    """
    
    def __init__(self, sigma_f: float = 1.0, length_scale: float = 0.3,
                 sigma_n: float = 0.01):
        self.log_params = np.log([sigma_f, length_scale, sigma_n])
        self.X_train = None
        self.y_train = None
        self.L_chol = None   # Facteur de Cholesky de (K + σ_n²I)
        self.alpha  = None   # (K + σ_n²I)⁻¹ y
    
    @property
    def sigma_f(self): return np.exp(self.log_params[0])
    @property
    def length_scale(self): return np.exp(self.log_params[1])
    @property
    def sigma_n(self): return np.exp(self.log_params[2])
    
    def rbf_kernel(self, X1: np.ndarray, X2: np.ndarray) -> np.ndarray:
        """Noyau RBF : k(x,x') = σ_f² exp(-||x-x'||²/(2ℓ²))"""
        X1, X2 = X1.reshape(-1,1), X2.reshape(-1,1)
        dists = ((X1 - X2.T)**2)
        return self.sigma_f**2 * np.exp(-dists / (2 * self.length_scale**2))
    
    def neg_log_marginal_likelihood(self, log_params: np.ndarray) -> float:
        """Log-vraisemblance marginale négative (pour minimisation)."""
        self.log_params = log_params
        n = len(self.y_train)
        K = self.rbf_kernel(self.X_train, self.X_train)
        C = K + (self.sigma_n**2 + 1e-6) * np.eye(n)
        try:
            L = np.linalg.cholesky(C)
        except np.linalg.LinAlgError:
            return 1e10
        alpha = np.linalg.solve(L.T, np.linalg.solve(L, self.y_train))
        # log p(y|X,θ) = -½ y^T α - Σ log(L_ii) - n/2 log(2π)
        nlml = 0.5 * self.y_train @ alpha + np.sum(np.log(np.diag(L))) + 0.5 * n * np.log(2*np.pi)
        return nlml
    
    def fit(self, X: np.ndarray, y: np.ndarray,
            optimize: bool = True, n_restarts: int = 5) -> 'GaussianProcessRegressor':
        """Entraîne le GP avec optimisation MLE-II des hyperparamètres."""
        self.X_train = X.copy()
        self.y_train = y.copy()
        
        if optimize:
            best_nlml = np.inf
            best_params = self.log_params.copy()
            rng = np.random.default_rng(42)
            for _ in range(n_restarts):
                x0 = rng.uniform(np.log(0.01), np.log(2.0), 3)
                result = minimize(self.neg_log_marginal_likelihood, x0,
                                  method='L-BFGS-B',
                                  bounds=[(-5, 2), (-5, 1), (-6, 0)])
                if result.fun < best_nlml:
                    best_nlml = result.fun
                    best_params = result.x.copy()
            self.log_params = best_params
        
        # Précalcul de la factorisation de Cholesky
        n = len(y)
        K = self.rbf_kernel(X, X)
        C = K + (self.sigma_n**2 + 1e-6) * np.eye(n)
        self.L_chol, _ = cho_factor(C, lower=True)
        self.alpha = cho_solve((self.L_chol, True), y)
        return self
    
    def predict(self, X_star: np.ndarray, return_std: bool = True) -> tuple:
        """
        Calcule la moyenne et variance a posteriori.
        μ*(x*) = k*^T α
        σ*²(x*) = k(x*,x*) - k*^T (K+σ_n²I)^{-1} k*
        """
        k_star = self.rbf_kernel(X_star, self.X_train)  # (M, n)
        mu = k_star @ self.alpha
        
        if return_std:
            v = cho_solve((self.L_chol, True), k_star.T)  # (n, M)
            var = self.sigma_f**2 - np.sum(k_star.T * v, axis=0)
            var = np.maximum(var, 1e-10)
            return mu, np.sqrt(var)
        return mu, None
    
    def estimate_pi(self, M: int = 10000) -> tuple:
        """Estime π et son incertitude bayésienne."""
        x_quad = np.linspace(0, 1, M)
        mu, sigma = self.predict(x_quad)
        dx = x_quad[1] - x_quad[0]
        # π̂ = 4 ∫ μ*(x)dx
        pi_hat = 4.0 * np.trapz(mu, x_quad)
        # σ_π ≈ 4√(∫ σ²*(x)dx) (borne conservative)
        pi_std = 4.0 * np.sqrt(np.sum(sigma**2) * dx)
        return pi_hat, pi_std


# ── Entraînement GP (sous-ensemble de n=200 points pour la rapidité) ─────────
# La complexité GP est O(n³), on utilise un sous-ensemble stratifié
N_GP = 200
idx_gp = np.linspace(0, N_TRAIN-1, N_GP, dtype=int)
X_gp, y_gp = X_train[idx_gp], y_train[idx_gp]

print("Entraînement du Processus Gaussien (MLE-II pour hyperparamètres)...")
t0 = time.time()
gp = GaussianProcessRegressor(sigma_f=1.0, length_scale=0.2, sigma_n=SIGMA)
gp.fit(X_gp, y_gp, optimize=True, n_restarts=8)
t_gp = time.time() - t0

pi_gp, pi_gp_std = gp.estimate_pi(M=10000)
print(f"\nHyperparamètres optimisés :")
print(f"  σ_f = {gp.sigma_f:.4f}  |  ℓ = {gp.length_scale:.4f}  |  σ_n = {gp.sigma_n:.5f}")
print(f"\nRésultats GP :")
print(f"  π̂_GP = {pi_gp:.10f}")
print(f"  Erreur absolue : {abs(pi_gp - PI):.2e}")
print(f"  IC bayésien (±2σ) : [{pi_gp - 2*pi_gp_std:.8f}, {pi_gp + 2*pi_gp_std:.8f}]")
print(f"  Temps : {t_gp:.2f}s")

In [ ]:
# ── Visualisation GP ──────────────────────────────────────────────────────────
mu_gp, std_gp = gp.predict(X_test)

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# (a) Postérieure GP avec bande d'incertitude
ax = axes[0]
ax.scatter(X_gp, y_gp, s=10, zorder=5, color=COLORS['gp'],
           alpha=0.6, label=f'Données ({N_GP} pts)')
ax.plot(X_test, y_exact, color=COLORS['true'], lw=2.5, label='$f(x)$ exacte')
ax.plot(X_test, mu_gp, color=COLORS['gp'], lw=2.5, label='Moyenne postérieure $\mu_*(x)$')
ax.fill_between(X_test, mu_gp - 2*std_gp, mu_gp + 2*std_gp,
                alpha=0.3, color=COLORS['gp'], label='IC 95% ($\pm 2\sigma_*$)')
ax.fill_between(X_test, mu_gp - std_gp, mu_gp + std_gp,
                alpha=0.4, color=COLORS['gp'])
ax.set_xlabel('$x$', fontsize=12); ax.set_ylabel('$y$', fontsize=12)
ax.set_title(f'GP Postérieure\n$\hat{{\\pi}}$ = {pi_gp:.6f} ± {pi_gp_std:.2e}', fontweight='bold')
ax.legend(fontsize=9, loc='lower left')
ax.set_xlim(-0.02, 1.02)

# (b) Incertitude postérieure σ*(x)
ax = axes[1]
ax.plot(X_test, std_gp, color=COLORS['gp'], lw=2)
ax.fill_between(X_test, 0, std_gp, alpha=0.3, color=COLORS['gp'])
ax.set_xlabel('$x$', fontsize=12)
ax.set_ylabel('$\sigma_*(x)$ (écart-type postérieur)', fontsize=11)
ax.set_title('Incertitude épistémique du GP\n(faible là où des données existent)', fontweight='bold')
ax.grid(True, alpha=0.3)
# Ajouter les positions des points d'entraînement
ax.scatter(X_gp, np.zeros_like(X_gp) - 2e-5, s=8, color=COLORS['gp'],
           alpha=0.5, zorder=5, label='Points d\'entraînement')
ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('fig_gp.png', bbox_inches='tight')
plt.show()

---
## 6. Comparaison Globale des Méthodes

Cette section effectue une analyse comparative rigoureuse selon trois critères :
1. **Précision** : erreur absolue $|\hat{\pi} - \pi|$
2. **Convergence** : taux empirique en fonction de $n$
3. **Coût computationnel** : temps CPU


In [ ]:
# ── Résumé des résultats ──────────────────────────────────────────────────────
results = {
    'Monte-Carlo\n(n=100k)': {
        'pi': pi_mc_final, 'error': abs(pi_mc_final - PI),
        'time': 0.015, 'color': COLORS['mc']
    },
    f'Poly. (d={poly_est.best_degree})': {
        'pi': poly_est.pi_hat, 'error': abs(poly_est.pi_hat - PI),
        'time': t_poly, 'color': COLORS['poly']
    },
    'MLP\n(3 couches, n=300)': {
        'pi': pi_mlp, 'error': abs(pi_mlp - PI),
        'time': t_mlp, 'color': COLORS['mlp']
    },
    'Processus\nGaussien': {
        'pi': pi_gp, 'error': abs(pi_gp - PI),
        'time': t_gp, 'color': COLORS['gp']
    },
}

# ── Tableau récapitulatif ─────────────────────────────────────────────────────
print("═" * 75)
print(f"{'Méthode':<30} {'π̂':>14} {'Erreur abs.':>14} {'Temps (s)':>10}")
print("═" * 75)
for name, r in results.items():
    name_clean = name.replace('\n', ' ')
    print(f"{name_clean:<30} {r['pi']:>14.10f} {r['error']:>14.2e} {r['time']:>10.3f}")
print("─" * 75)
print(f"{'π (valeur exacte)':<30} {PI:>14.10f}")
print("═" * 75)

In [ ]:
# ── Convergence comparative ───────────────────────────────────────────────────
ns_comp = [50, 100, 200, 400, 1000, 2000, 5000]
N_SEEDS = 5  # Moyenne sur plusieurs seeds pour la robustesse

all_errors = {'mc': [], 'poly': [], 'gp': []}

for n in ns_comp:
    errs = {'mc': [], 'poly': [], 'gp': []}
    for seed in range(N_SEEDS):
        # Monte-Carlo
        pi_hat_mc, _, _, _ = monte_carlo_pi(n * 10, seed=seed)  # MC a besoin de plus de points
        errs['mc'].append(abs(pi_hat_mc - PI))
        
        # Poly
        Xp, yp, _ = generate_dataset(n, SIGMA, seed=seed)
        try:
            ep = PolynomialPiEstimator(max_degree=10, alpha=1e-12)
            ep.fit(Xp, yp)
            errs['poly'].append(abs(ep.pi_hat - PI))
        except:
            errs['poly'].append(np.nan)
        
        # GP (sous-ensemble)
        n_gp_comp = min(n, 150)
        Xg, yg, _ = generate_dataset(n_gp_comp, SIGMA, seed=seed)
        try:
            gp_comp = GaussianProcessRegressor(sigma_f=1.0, length_scale=0.2, sigma_n=SIGMA)
            gp_comp.fit(Xg, yg, optimize=False)  # Sans optim pour la rapidité
            pi_g, _ = gp_comp.estimate_pi(M=5000)
            errs['gp'].append(abs(pi_g - PI))
        except:
            errs['gp'].append(np.nan)
    
    for k in all_errors:
        arr = np.array(errs[k])
        all_errors[k].append(np.nanmean(arr))

all_errors = {k: np.array(v) for k, v in all_errors.items()}

# ── Figure finale : comparaison ───────────────────────────────────────────────
fig = plt.figure(figsize=(15, 10))
gs  = gridspec.GridSpec(2, 3, figure=fig, hspace=0.4, wspace=0.35)

# (1) Convergence log-log
ax1 = fig.add_subplot(gs[0, :])
ns_mc = [n*10 for n in ns_comp]
ax1.loglog(ns_mc, all_errors['mc'], 'o-', color=COLORS['mc'], lw=2, ms=7,
           label='Monte-Carlo')
ax1.loglog(ns_comp, all_errors['poly'], 's-', color=COLORS['poly'], lw=2, ms=7,
           label='Régression polynomiale')
ax1.loglog(ns_comp, all_errors['gp'], 'd-', color=COLORS['gp'], lw=2, ms=7,
           label='Processus Gaussien')

# Références théoriques
ref_x = np.array([50, 50000])
ax1.loglog(ref_x, 3.0 * ref_x**(-0.5), '--', color=COLORS['mc'], alpha=0.5, lw=1.5,
           label=r'Réf. $O(n^{-1/2})$')
ax1.loglog(ref_x[0:1]*np.array([1, 100]),
           [all_errors['poly'][0], all_errors['poly'][0]*0.001],
           '--', color=COLORS['poly'], alpha=0.5, lw=1.5, label=r'Réf. $O(n^{-1.5})$')
ax1.set_xlabel('$n$ (taille du dataset / nbre tirages MC)', fontsize=12)
ax1.set_ylabel('Erreur absolue $|\hat{\\pi}-\pi|$', fontsize=12)
ax1.set_title('Comparaison des taux de convergence (log-log)', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10, loc='upper right', ncol=2)
ax1.grid(True, alpha=0.3, which='both')
ax1.axhline(1e-4, color='gray', lw=0.8, alpha=0.5, linestyle=':')
ax1.text(50, 1.2e-4, 'Seuil $10^{-4}$', fontsize=8, color='gray')

# (2) Barres d'erreur finale
ax2 = fig.add_subplot(gs[1, 0])
labels = [k.replace('\n', ' ') for k in results.keys()]
errors_final = [r['error'] for r in results.values()]
colors_bar   = [r['color'] for r in results.values()]
bars = ax2.bar(range(len(labels)), errors_final, color=colors_bar, alpha=0.8, edgecolor='white')
ax2.set_yscale('log')
ax2.set_xticks(range(len(labels)))
ax2.set_xticklabels(labels, fontsize=8)
ax2.set_ylabel('$|\hat{\\pi} - \pi|$ (log)', fontsize=10)
ax2.set_title('Erreur absolue finale', fontweight='bold')
for bar, err in zip(bars, errors_final):
    ax2.text(bar.get_x() + bar.get_width()/2., bar.get_height()*1.2,
             f'{err:.1e}', ha='center', va='bottom', fontsize=7.5)
ax2.grid(True, alpha=0.3, axis='y')

# (3) Frontière Pareto précision / temps
ax3 = fig.add_subplot(gs[1, 1])
for name, r in results.items():
    n_clean = name.replace('\n', ' ')
    ax3.scatter(r['time'], r['error'], s=120, color=r['color'],
                zorder=5, edgecolors='white', linewidths=1.5)
    ax3.annotate(n_clean, (r['time'], r['error']),
                 textcoords='offset points', xytext=(5, 5), fontsize=7.5)
ax3.set_xscale('log'); ax3.set_yscale('log')
ax3.set_xlabel('Temps CPU (s, log)', fontsize=10)
ax3.set_ylabel('$|\hat{\\pi} - \pi|$ (log)', fontsize=10)
ax3.set_title('Frontière de Pareto\nPrécision vs Coût', fontweight='bold')
ax3.grid(True, alpha=0.3, which='both')

# (4) Distributions des estimateurs
ax4 = fig.add_subplot(gs[1, 2])
estimates = {
    'MC': [monte_carlo_pi(1000, seed=s)[0] for s in range(200)],
    'Poly': [],
    'GP': [],
}
for seed in range(50):
    Xp, yp, _ = generate_dataset(300, SIGMA, seed=seed)
    try:
        ep = PolynomialPiEstimator(max_degree=10, alpha=1e-12)
        ep.fit(Xp, yp)
        estimates['Poly'].append(ep.pi_hat)
    except: pass
    try:
        gp_s = GaussianProcessRegressor(sigma_f=1.0, length_scale=0.2, sigma_n=SIGMA)
        gp_s.fit(Xp[:100], yp[:100], optimize=False)
        pi_g_s, _ = gp_s.estimate_pi(M=2000)
        estimates['GP'].append(pi_g_s)
    except: pass

for key, col, lw in zip(['MC', 'Poly', 'GP'],
                         [COLORS['mc'], COLORS['poly'], COLORS['gp']],
                         [1.5, 2, 2]):
    vals = np.array(estimates[key])
    ax4.hist(vals, bins=30, density=True, alpha=0.5, color=col,
             edgecolor='white', label=key)
ax4.axvline(PI, color='k', lw=2, linestyle='--', label=f'π = {PI:.4f}')
ax4.set_xlabel('$\hat{\\pi}$', fontsize=10)
ax4.set_ylabel('Densité', fontsize=10)
ax4.set_title('Distribution des estimateurs\n(variance et biais)', fontweight='bold')
ax4.legend(fontsize=9)

plt.suptitle('Analyse Comparative : Approximation de $\pi$ par Machine Learning',
             fontsize=14, fontweight='bold', y=1.01)
plt.savefig('fig_comparaison.png', bbox_inches='tight', dpi=150)
plt.show()

print("Figure de comparaison sauvegardée.")

---
## 7. Étude Théorique : Complexité de Rademacher

Cette section calcule et visualise la **complexité de Rademacher empirique** $\hat{\mathfrak{R}}_n(\mathcal{H})$ pour la classe polynomiale, qui borne l'erreur de généralisation (Bartlett & Mendelson, 2002) :
$$\mathcal{R}(f) \leq \hat{\mathcal{R}}_n(f) + 2\hat{\mathfrak{R}}_n(\mathcal{H}) + \sqrt{\frac{\log(1/\delta)}{2n}}$$


In [ ]:
def rademacher_complexity(X: np.ndarray, degree: int,
                           n_mc: int = 500, seed: int = 0) -> float:
    """
    Estime la complexité de Rademacher empirique pour la classe polynomiale.
    
    𝔑̂_n(H) = E_σ[sup_{||θ||≤1} (1/n) Σ_i σ_i p_θ(x_i)]
    Pour les polynômes normalisés, cela se réduit à (1/n) ||Φ||_F / √(d+1)
    """
    rng = np.random.default_rng(seed)
    n   = len(X)
    # Matrice de features de Vandermonde normalisée
    Phi = np.column_stack([X**k for k in range(degree+1)])  # (n, d+1)
    
    # Estimation Monte-Carlo : E[sup_{||θ||_2<=1} (1/n) σ^T Φ θ]
    # = E[(1/n) ||Φ^T σ||_2] (le sup est atteint par θ=Φ^Tσ/||Φ^Tσ||)
    sups = []
    for _ in range(n_mc):
        sigma = rng.choice([-1., 1.], size=n)
        s = np.linalg.norm(Phi.T @ sigma) / n
        sups.append(s)
    return np.mean(sups)


degrees = range(1, 14)
ns_rad  = [50, 200, 500]

fig, axes = plt.subplots(1, 2, figsize=(13, 5))

# (a) Rademacher vs degré pour différents n
ax = axes[0]
for n, ls in zip(ns_rad, ['-', '--', ':']):
    X_rad, _, _ = generate_dataset(n, 0, seed=42)
    rad = [rademacher_complexity(X_rad, d, n_mc=200) for d in degrees]
    ax.plot(list(degrees), rad, ls, lw=2, label=f'$n={n}$')

ax.set_xlabel('Degré $d$', fontsize=11)
ax.set_ylabel(r'$\hat{\mathfrak{R}}_n(\mathcal{P}_d)$', fontsize=11)
ax.set_title('Complexité de Rademacher\nvs degré polynomial', fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# (b) Borne de généralisation PAC
ax = axes[1]
delta = 0.05
n_vals = [50, 100, 200, 500, 1000]
d_fixed = 7
pac_bounds = []
for n in n_vals:
    X_b, y_b, _ = generate_dataset(n, SIGMA, seed=42)
    pipe = Pipeline([('poly', PolynomialFeatures(degree=d_fixed)),
                     ('ridge', Ridge(alpha=1e-12))])
    pipe.fit(X_b.reshape(-1,1), y_b)
    y_pred_b = pipe.predict(X_b.reshape(-1,1))
    emp_risk = np.mean((y_pred_b - y_b)**2)
    
    rad = rademacher_complexity(X_b, d_fixed, n_mc=100)
    conf_term = np.sqrt(np.log(1/delta) / (2*n))
    pac_bound = emp_risk + 2*rad + conf_term
    pac_bounds.append({'n': n, 'emp': emp_risk, 'rad': rad, 'conf': conf_term, 'bound': pac_bound})

n_plot = [p['n'] for p in pac_bounds]
ax.loglog(n_plot, [p['bound'] for p in pac_bounds], 'o-', color=COLORS['poly'],
          lw=2, ms=8, label='Borne PAC totale')
ax.loglog(n_plot, [p['emp'] for p in pac_bounds], 's--', color=COLORS['mc'],
          lw=2, ms=6, label='Risque empirique $\hat{R}_n$')
ax.loglog(n_plot, [2*p['rad'] for p in pac_bounds], '^--', color=COLORS['gp'],
          lw=2, ms=6, label=r'$2\hat{\mathfrak{R}}_n$')
ax.loglog(n_plot, [p['conf'] for p in pac_bounds], 'v--', color=COLORS['mlp'],
          lw=2, ms=6, label=r'Terme de conf. $\sqrt{\log(1/\delta)/2n}$')

ax.set_xlabel('$n$', fontsize=11)
ax.set_ylabel('Valeur (log)', fontsize=11)
ax.set_title(f'Décomposition de la borne PAC ($d={d_fixed}$, $\delta={delta}$)', fontweight='bold')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3, which='both')

plt.tight_layout()
plt.savefig('fig_rademacher.png', bbox_inches='tight')
plt.show()
print("Analyse théorique (Rademacher) complétée.")

---
## 8. Conclusion et Synthèse

| Méthode | Hypothèses | Taux (théorique) | Forces | Limites |
|---|---|---|---|---|
| Monte-Carlo | Aucune (universelle) | $O(n^{-1/2})$ | Simple, parallelisable | Lente convergence |
| Poly. OLS | $f$ analytique | $O(\rho^{-d})$ en degré | Intégration exacte | Conditionnement de Vandermonde |
| MLP (Adam) | $f \in$ classe de Barron | $O(m^{-1})$ en neurones | Flexible, haute précision | Non-convexe, opaque |
| GP (RBF) | $f \in$ RKHS de $k$ | Expo. en régularité | Incertitude quantifiée | Cubique en $n$ |


In [ ]:
# ── Résumé final ──────────────────────────────────────────────────────────────
print("╔═══════════════════════════════════════════════════════════════════════╗")
print("║       APPROXIMATION DE π PAR MACHINE LEARNING — BILAN FINAL          ║")
print("╠═══════════════════════════════════════════════════════════════════════╣")
print(f"║  Valeur de référence : π = {PI:<45.15f}║")
print("╠═══════════════════════════════════════════════════════════════════════╣")

methodes = [
    ('Monte-Carlo (n=100k)',     pi_mc_final),
    (f'Poly. (d={poly_est.best_degree}, n=300)',  poly_est.pi_hat),
    ('MLP (n=300, 3000 ep.)',    pi_mlp),
    ('GP (n=200)',               pi_gp),
]

for name, pi_est in methodes:
    err = abs(pi_est - PI)
    n_correct = max(0, -int(np.floor(np.log10(err))) - 1) if err > 0 else 15
    print(f"║  {name:<28} π̂ = {pi_est:.10f}  err={err:.2e}  ║")

print("╠═══════════════════════════════════════════════════════════════════════╣")
print("║  VAINQUEUR (erreur minimale) : Processus Gaussien                    ║")
print("║  VAINQUEUR (rapport précision/temps) : Régression Polynomiale        ║")
print("║  Référence : Leibniz-Gregory : 4*(1-1/3+1/5-...) → O(k^{-1})        ║")
print("╚═══════════════════════════════════════════════════════════════════════╝")

# Série de Leibniz pour comparaison
k_vals = np.arange(0, 100_000)
leibniz = 4 * np.sum((-1)**k_vals / (2*k_vals + 1))
print(f"\nSérie de Leibniz (100k termes) : π̂ = {leibniz:.10f}, err = {abs(leibniz-PI):.2e}")